In [26]:
import pywikibot
from pywikibot import pagegenerators
from tqdm import tqdm
import re
import requests


lang_wiki = pywikibot.Site(code='en', fam='hitchwiki')
if not lang_wiki.user():
    lang_wiki.login()

In [27]:
places = {}

In [28]:
pages = list(pagegenerators.AllpagesPageGenerator(site=lang_wiki))

In [29]:
for page in tqdm(pages):
    try:
        if any(s in page.text for s in ["{{Infobox"]):
            places[page.title()] = {"text": page.text}
    except Exception as e:
        print(f"Error processing page: {e}")
        continue

 92%|█████████▏| 5034/5456 [11:09<00:44,  9.42it/s]

Error processing page: Invalid AutoFamily('hitchwiki.org')


100%|██████████| 5456/5456 [11:58<00:00,  7.59it/s]


In [30]:
len(places)

765

In [55]:
places

{'A62 (France)': {'text': '{{Infobox Location\n|country = France\n|map = <map lat=\'44.253048694828\' lng=\'0.46269589401237\' zoom=\'8\' view=\'0\' />\n}}\nThe \'\'\'A62\'\'\' is a motorway in [[France]], it leads from [[Bordeaux]] to [[Toulouse]].\n\n== Hitching the \'\'\'A62\'\'\' ==\n\n{{legend}}\n\n{| class="wikitable"\n|- \n! style="width: 30em" | !! style="width:30em" | !! style="width: 30em" | \n|-\n| <p align="center">{{south}}</p> ||  || <p align="center">{{north}}</p>\n|-\n|  || {{junction}} {{AutorouteFR|630}} ||  |\n|-\n|  || (1) {{ramp}} Martillac ||  |\n|-\n|  || (1.1) {{ramp}} La Brède ||  |\n|-\n|  || {{peage}} ||  |\n|-\n| {{gas}} {{rest}} service station [[Les Landes Sud]] ||  || {{gas}} service station [[Les Landes Nord]]\n|-\n|  || (2) {{ramp}} Podensac ||  |\n|-\n|  || (3) {{ramp}} Langon ||  |\n|-\n|  || {{junction}} {{AutorouteFR|65}} ||  |\n|-\n|  || (4) {{ramp}} La Réole ||  |\n|-\n|  || (5) {{ramp}} Marmande ||  |\n|-\n|  || {{gas}} service station [[Mas d\'A

In [ ]:
for place, items in tqdm(places.items()):
    try:
        places[place]["urls"] = []
        if items["text"]:
            matches = re.findall(r'http[s]?://[^\s\]\)]+', items["text"])
            urls = [url for url in matches if any(x in url for x in ['map', 'lat', 'lon', 'osm', 'wego'])]
            places[place]["urls"] = urls
    except Exception as e:
        print(f"Error processing page {place}: {e}")
        continue


100%|██████████| 765/765 [00:00<00:00, 40063.46it/s]


In [42]:
for place, items in tqdm(places.items()):
    try:
        coords = []
        places[place]["coords"] = coords
        if "urls" in items:
            for url in items["urls"]:
                response = requests.head(url, allow_redirects=True, timeout=5)
                if response.status_code < 400:
                    # collection of points
                    if "viewer" in response.url:
                        continue
                    
                    pattern = r"(-?\d+\.\d{4,}),(-?\d+\.\d{4,})"
                    matches = re.findall(pattern, response.url)
                    if len(matches) == 0:
                        print(f"No coordinates found in {url}")
                        continue
                    if len(matches) > 1:
                        print(
                            f"Multiple coordinates found in {url} likely a path: {matches}"
                        )
                        continue
                    lat, lon = matches[0]
                    coords.append((url, response.url, (lat, lon)))
        places[place]["coords"] = coords
    except Exception as e:
        print(f"Error processing page {place}: {e}")
        continue

  1%|          | 7/765 [00:00<00:53, 14.14it/s]

Error processing page Adelaide: HTTPConnectionPool(host='timetables.adelaidemetro.com.au', port=80): Max retries exceeded with url: /timetables/routemaps/401-403-900_270909_routemap.pdf (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7fd86c495d30>: Failed to resolve 'timetables.adelaidemetro.com.au' ([Errno -2] Name or service not known)"))


  2%|▏         | 13/765 [00:01<01:08, 10.95it/s]

No coordinates found in http://www.aims.org.af/maps/national/routh_maps/road_network/road_network.pdf
Multiple coordinates found in https://wego.here.com/directions/walk/Pra%C3%A7a-Sacadura-Cabral-46,-4630-219-Marco-de-Canaveses,-Portugal:41.18567,-8.14939/REPSOL,-Avenida-dos-Bombeiros-Volunt%C3%A1rios-do-Marco,-4630-470-Marco-de-Canaveses:620ez64q-b1bb2e884140423cab7e9393743fd2f4?map=41.18607,-8.15076,17,pedestrian&msg=Pra%C3%A7a%20Sacadura%20Cabral%2046 likely a path: [('41.18567', '-8.14939'), ('41.18607', '-8.15076')]
No coordinates found in https://www.openstreetmap.org/directions?engine=mapzen_foot&route=41.1867%2C-8.1552%3B41.1938%2C-8.1546#map=17/41.18914/-8.15181
No coordinates found in https://www.openstreetmap.org/directions?engine=mapzen_car&route=41.16165%2C-8.12779%3B41.16643%2C-8.58833#map=17/41.16577/-8.58538
No coordinates found in https://www.openstreetmap.org/directions?engine=mapzen_car&route=41.16165%2C-8.12779%3B40.64050%2C-8.65378#map=9/40.9401/-8.3949
No coordin

  7%|▋         | 52/765 [00:02<00:35, 20.04it/s]

...

 98%|█████████▊| 748/765 [02:35<00:03,  4.43it/s]

Multiple coordinates found in http://maps.google.com/?q=zagreb&ie=UTF8&hq=&hnear=City+of+Zagreb,+Croatia&ll=45.874518,15.829003&spn=0.009815,0.02459&t=h&z=16 likely a path: [('45.874518', '15.829003'), ('0.009815', '0.02459')]
Multiple coordinates found in https://maps.google.pl/maps?q=51.935587,15.50573&ll=51.937717,15.548337&spn=0.012184,0.033023&num=1&t=m&z=16&layer=c&cbll=51.937881,15.548287&panoid=kjqcj1JjoZbntxi_yfarNQ&cbp=12,347.7,,0,4.3 likely a path: [('51.935587', '15.50573'), ('51.937717', '15.548337'), ('0.012184', '0.033023'), ('51.937881', '15.548287')]
Multiple coordinates found in https://maps.google.pl/maps?q=51.935587,15.50573&ll=51.937717,15.548337&spn=0.012184,0.033023&num=1&t=m&z=16&layer=c&cbll=51.937881,15.548287&panoid=kjqcj1JjoZbntxi_yfarNQ&cbp=12,347.7,,0,4.3 likely a path: [('51.935587', '15.50573'), ('51.937717', '15.548337'), ('0.012184', '0.033023'), ('51.937881', '15.548287')]


 98%|█████████▊| 751/765 [02:36<00:03,  3.65it/s]

Multiple coordinates found in https://maps.google.pl/?ll=51.957629,15.532898&spn=0.006089,0.016512&t=m&z=17&layer=c&cbll=51.957576,15.533344&panoid=BLwKAfBO_kB24FyRyafI5g&cbp=12,127.3,,0,5.5 likely a path: [('51.957629', '15.532898'), ('0.006089', '0.016512'), ('51.957576', '15.533344')]


100%|██████████| 765/765 [02:37<00:00,  4.86it/s]


In [48]:
for place, items in tqdm(places.items()):
    try:
        places[place].pop("new_text", None)
        if "coords" in items and items["coords"]:
            for url, real_url, (lat, lon) in items["coords"]:
                if url in items["text"]:
                    escaped_url = re.escape(url)
                    pattern = rf'\[{escaped_url}\s+([^\]]+)\]'
                    match = re.search(pattern, items["text"])
                    if match:
                        link_text = match.group(1)
                        new_text = items["text"].replace(
                            f"[{url} {link_text}]",
                            f"''{link_text}'' {{{{Coords|{lat}|{lon}|1}}}}"
                        )
                    else:
                        new_text = items["text"].replace(
                            url,
                            f"{{{{Coords|{lat}|{lon}}}}}"
                        )
                    places[place]["new_text"] = new_text
    except Exception as e:
        print(f"Error updating page {place}: {e}")
        continue

100%|██████████| 765/765 [00:00<00:00, 60358.21it/s]


In [49]:
edits = {k: v for k, v in places.items() if "new_text" in v}

In [54]:
for place, items in tqdm(list(edits.items())[10:]):
    try:
        page = pywikibot.Page(lang_wiki, place)
        if page.text != items["new_text"]:
            page.text = items["new_text"]
            page.save(summary="Adding coordinates from map URLs", minor=False)
            print(f"Updated: {place}")
        else:
            print(f"No changes for {place}")
    except Exception as e:
        print(f"Error saving page {place}: {e}")
        continue

  3%|▎         | 1/32 [00:02<01:07,  2.18s/it]

Updated: Firenze


Sleeping for 7.9 seconds, 2025-05-25 00:58:29
Page [[hitchwiki:Glasgow]] saved
  6%|▋         | 2/32 [00:12<03:20,  6.68s/it]

Updated: Glasgow


Sleeping for 8.1 seconds, 2025-05-25 00:58:39
Page [[hitchwiki:Graz]] saved
  9%|▉         | 3/32 [00:21<03:55,  8.12s/it]

...

Updated: Wrocław


Sleeping for 8.1 seconds, 2025-05-25 01:04:09
Page [[hitchwiki:Zagreb]] saved
 97%|█████████▋| 31/32 [05:52<00:10, 10.09s/it]

Updated: Zagreb


Sleeping for 7.6 seconds, 2025-05-25 01:04:20
Page [[hitchwiki:Zürich]] saved
100%|██████████| 32/32 [06:03<00:00, 11.35s/it]

Updated: Zürich


In [52]:
edits

{'Ancede (Baião)': {'text': "__NOTOC__\n{{Infobox Italian Location\n|country = Portugal \n|state = North Region\n|district = Porto\n|map = <map lat='41.104671' lng='-8.061416' zoom='11' view='3' float='right' />\n|pop = 53,450\n|plate = -\n|motorways = A4, N211\n|hitch = {{average}}\n}}\n'''Ancede''' is a parish in [[Portugal]], belonging to '''Baiao''' municipality, where the [http://www.festanca.org Festança] will take place in the summer of 2021.\n\n== Hitchhiking out ==\n=== West towards [[Porto]] ===\n\n==== Option 1: Repsol Gas Station ====\n\nIt should be pretty easy and direct to get to Porto, as many people drive to Porto from Marco de Canaveses everyday. Just before the river Rio Tamega, one of the two main rivers of Marco de Canaveses, you will have this [https://wego.here.com/portugal/marco-de-canaveses/petrol-station/repsol--620ez64q-b1bb2e884140423cab7e9393743fd2f4?x=ep&map=41.18695,-8.15549,15,normal gas station]. You can easily walk to the gas station located in the Ave

In [47]:
stop

NameError: name 'stop' is not defined

In [ ]:
links = []

for place in tqdm(places):
    page = pywikibot.Page(lang_wiki, place)

    pattern = r"(https://osm\.org/go/[^\s\]\)]+|https://www\.openstreetmap\.org/[^\s\]\)]+|https://www\.openstreetmap\.org/node/[^\s\]\)]+|https://goo\.gl/maps/[^\s\]\)]+|https://hitchmap\.com/[^\s\]\)]+|http://maps\.google\.be/maps[^\s\]\)]+|http://maps\.google\.com/maps[^\s\]\)]+|https://www\.google\.co\.uk/maps/[^\s\]\)]+|https://wego\.here\.com/[^\s\]\)]+)"
    matches = re.findall(pattern, page.text)

    if matches:
        t = (place, matches)
        print(t)
        links.append(t)
        

    # page = pywikibot.Page(en_wiki, eng_page_name)
    # page.text = f"{{{{ai-enhanced}}}}\n{translated_page}"
    # page.save(f"Create this article by translation with language model (gpt 4.1) from hitchwiki original language: {lang} {ru_page_name}")
    # print(page.title())
    # print("******")

In [ ]:
locations